In [1]:
# nobody knows anything
#                  -Kai
#
# github.com/kaibrooks

In [48]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [49]:
# doin thangs
epochs=10 #60

In [50]:
path = get_file(
    'drseuss.txt',
    origin='data/drseuss.txt')

with io.open(path, encoding='utf-8', errors='ignore') as f: # errors=ignore strips non utf-8 chars
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 30504


In [51]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorizing...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Done')

total chars: 38
nb sequences: 10155
Vectorization...
Done


In [44]:
# model
print('Building model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done')

Building model...
Done


In [45]:
# train me
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # print text at each epoch
    print()
    print('****************************** Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('--------------- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--------------- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=epochs,
          callbacks=[print_callback])

Epoch 1/10
10155/10155 [==============================] - 8s 766us/step - loss: 2.7928

------------------------------ Generating text after Epoch: 0
--------------- diversity: 0.2
--------------- Generating with seed: "t.
oh, the things they will bump!
oh, th"
t.
oh, the things they will bump!
oh, the the the hee there hee the hee the hee hee see the there he the hee cae se the hee soue se see the toue the hee thee see there wee see coue the thee there le hee see the hee fe there wee the the hee the wee there there he he the hee see hee he wee there the the hee the hee here the the hee the soue the he the hee the hee the hee the hee  hee here the hee see he he the hee le soue coue there the t
--------------- diversity: 0.5
--------------- Generating with seed: "t.
oh, the things they will bump!
oh, th"
t.
oh, the things they will bump!
oh, there thee thet heye.
rere he touc de am the toee d the there ee ee se .
e lere the take whep then ane theree coue heos se heee use boue the san.
t

"lould fo row do not like them and a do not like thes a good the paddle say sime i this is a four foo do not like the foos say.
"hould the ghan ho shee lowis whall sin the came i co house the same an he said the foo
so ho sove 
--------------- diversity: 1.0
--------------- Generating with seed: "head wasn't screwed on just right.
it co"
head wasn't screwed on just right.
it cool fish ! a woold will shen be man licks a hour so at thing soo yishe aik nothere up bith"
woull quat.
here ar my thing.
"ar sowllr the saing oo lould d an a sood no
.
not like a ho not lig s as the fo roxse sany un wo like of cat i i youn go rooke should croe lholes bouse, would the a bix.
i ho not loke eo hear your a six.
hall shis a paw 
and..
beo ros, lould y theg aw.
and a do
not el i that is
--------------- diversity: 1.2
--------------- Generating with seed: "head wasn't screwed on just right.
it co"
head wasn't screwed on just right.
it comr i comes oue pox hes, as chow a noee in "heist!"",ray whe save ha

"i do not lek at like ham!
this woy's with a mreey toluit playous wey, and shor.
my eant! un up all flumm grir.
you racksoey, ul siet fwhis wey s ohey! we sawineas fibe ar so what we par shou h?
but?

and the fop of thery ar a wpukeduut ju
Epoch 8/10
10155/10155 [==============================] - 8s 756us/step - loss: 1.0961

------------------------------ Generating text after Epoch: 7
--------------- diversity: 0.2
--------------- Generating with seed: "s not all!
oh, no.
that is not all...

""
s not all!
oh, no.
that is not all...

"well the tare the the sat.
stoe the the tare.
shere is why was same the the wast puppup.
we saw a sare the the was out of the the theet theetle beetle but.
shen we saw the the tads wet a not.
i do not like them in the pleetles the they, so the the sat.
shere are the the treet they the tood the treetle beetle but.

some and the tats in the house.
i do not like them in the ray.

sooe the the the the
--------------- diversity: 0.5
--------------- Generating